In [ ]:
import src

from src.runner import Runner
device = 'cuda:0'


In [3]:
loading_cfg = dict(
    batch_size=512,
    num_workers=4,
)

eurosat_cfg = dict(
    type='EuroSATDataset',
    transform=[
        dict(type='Resize', size=(128, 128)),
        dict(type='ToTensor'),
    ]
)

imagenet_cfg = dict(
    type='ImageNetDataset',
    transform=[
        dict(type='Resize', size=(128, 128)),
        dict(type='ToTensor'),
    ]
)

optim_cfg = dict(
    type='Adam',
    lr=0.001,
    weight_decay=1e-4,
)

backbone_cfg = dict(
    type='ResNet',
    idims=3,
    odims=64,
    base_dims=12,
    arch=[2, 2, 2, 2],
    dropout=0.2,
)

In [ ]:
# -----------------------------------
# Vanilla EuroSAT model configuration
# -----------------------------------

eurosat_model_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=10,  # EuroSAT has 10 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    )
)

eurosat_vanilla = Runner(model=eurosat_model_cfg, dataloader_cfg=loading_cfg, dataset=eurosat_cfg, optim=optim_cfg, device=device, work_dir='results/eurosat_vanilla')
eurosat_vanilla.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

# -----------------------------------
# Tiny ImageNet model configuration
# -----------------------------------
tiny_imnet_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=200,  # Tiny ImageNet has 200 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    )
)

tiny_imnet = Runner(model=tiny_imnet_cfg, dataloader_cfg=loading_cfg, dataset=imagenet_cfg, optim=optim_cfg, device=device, work_dir='results/tiny_imnet')
tiny_imnet.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)


# -----------------------------------
# TFL EuroSAT model configuration
# -----------------------------------
tfl_eurosat_model_cfg = dict(
    type='EuroSATModel',
    backbone_cfg=backbone_cfg,
    head_cfg=dict(
        type='FFN',
        idims=64,
        odims=10,  # EuroSAT has 10 classes
        hidden_dims=1024,
        nlayers=6,
        dropout=0.2,
    ),
    ckpt=dict(
        path = tiny_imnet.best_model_path,
        load_head=False,
        load_backbone=True,
        strict=True,
    )
)

tfl_eurosat = Runner(model=tfl_eurosat_model_cfg, dataloader_cfg=loading_cfg, dataset=eurosat_cfg, optim=optim_cfg, device=device, work_dir='results/tfl_eurosat')
tfl_eurosat.run(mode='train', val_interval=1, log_interval=1, epochs=100, start_epoch=1)

No seed provided, using initial seed: 176196775
     Model Parameters Summary     
----------------------------------
Trainable params     | 5,857,878
Non-trainable params |         0
Total params         | 5,857,878
- Backbone params    | 1,582,668
- Head params        | 4,275,210
Epoch 001/100 | Iter 37/37 | [██████████] |  | loss: 1.4146 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100.00% | val_loss: 3.8335 --> F1: 0.1076 | mAP: 0.1844
Model saved to results/eurosat_vanilla/run_20250613-060023/best_model.pth
Best model saved at epoch 1 with val_loss 2.2807
Epoch 002/100 | Iter 37/37 | [██████████] |  | loss: 1.0289 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>>>>>>] |  100.00% | val_loss: 1.1643 --> F1: 0.4817 | mAP: 0.6043
Model saved to results/eurosat_vanilla/run_20250613-060023/best_model.pth
Best model saved at epoch 2 with val_loss 1.9861
Epoch 003/100 | Iter 37/37 | [██████████] |  | loss: 0.8018 | lr: 1.0000e-03
Evaluating | Iter 5400/5400 | [>>>>>